In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import requests
import numpy as np
import folium
from IPython.display import display

# Load addresses from Excel file
file_path = 'Adressen.xlsx'
print("Lade Adressen aus Excel-Datei...")
addresses_df = pd.read_excel(file_path)

# Step 1: Geocoding addresses to get coordinates
print("Geocoding der Adressen beginnt...")
geolocator = Nominatim(user_agent="tsp_solver")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_coordinates(address):
    try:
        location = geocode(address)
        if location:
            print(f"Adresse '{address}' geokodiert: ({location.latitude}, {location.longitude})")
            return location.latitude, location.longitude
        else:
            print(f"Keine Koordinaten gefunden für: {address}")
            return None
    except Exception as e:
        print(f"Fehler beim Geocoding der Adresse '{address}': {e}")
        return None

addresses_df['Coordinates'] = addresses_df['Adresse'].apply(get_coordinates)
addresses_df.dropna(subset=['Coordinates'], inplace=True)
coordinates = addresses_df['Coordinates'].tolist()
print("Geocoding abgeschlossen.")

# Step 2: Function to create distance matrix using OSRM
def get_osrm_distance_matrix(coords):
    print(f"Rufe OSRM-Entfernungs- und Zeitmatrix ab für {len(coords)} Adressen...")
    coord_str = ";".join([f"{lon},{lat}" for lat, lon in coords])
    url = f"http://router.project-osrm.org/table/v1/driving/{coord_str}?annotations=distance"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        distance_matrix = np.array(data["distances"]) / 1000  # Convert meters to kilometers
        print("OSRM-Entfernungs- und Zeitmatrix erfolgreich abgerufen.")
        return distance_matrix
    else:
        print("Fehler beim Abrufen der OSRM-Daten.")
        raise Exception("Error fetching data from OSRM.")

# Step 3: Optimize route with TSP
def optimize_route_with_tsp(coords):
    distance_matrix = get_osrm_distance_matrix(coords)
    
    data = {
        'distance_matrix': distance_matrix,
        'num_vehicles': 1,
        'depot': 0
    }
    
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(data['distance_matrix'][from_node][to_node] * 1000)

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    solution = routing.SolveWithParameters(search_parameters)

    ordered_coords = []
    if solution:
        index = routing.Start(0)
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            ordered_coords.append(coords[node_index])
            index = solution.Value(routing.NextVar(index))
        ordered_coords.append(coords[0])  # Complete the loop

    return ordered_coords

# Step 4: Solve TSP and visualize the route
print("Löse TSP für die optimale Rundroute...")
optimized_route = optimize_route_with_tsp(coordinates)
print("Optimierte Route berechnet.")

# Print the ordered route
print("\nOptimierte Route:")
for coord in optimized_route:
    address = addresses_df.loc[addresses_df['Coordinates'] == coord, 'Adresse'].values[0]
    print(f"{address} - Koordinaten: {coord}")

# Visualize the optimized route on a map
if optimized_route:
    print("Visualisiere die optimierte Route auf der Karte...")
    start_coords = optimized_route[0]
    map_ = folium.Map(location=start_coords, zoom_start=12)

    # Add markers and polyline to show the route order
    for coord in optimized_route:
        address = addresses_df.loc[addresses_df['Coordinates'] == coord, 'Adresse'].values[0]
        folium.Marker(location=coord, tooltip=address).add_to(map_)
    
    folium.PolyLine(locations=optimized_route, color="blue", weight=2.5, opacity=1).add_to(map_)
    display(map_)
    print("Karte wurde erfolgreich erstellt.")


Output here...
